# Imports

In [ ]:
from dotenv import load_dotenv
import logging

load_dotenv(dotenv_path=".env", verbose=True, override=True)
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import os

from autocommit.core.enums import EnvironmentKey
from autocommit_evaluation.cmg.evaluators import CommitMessageGenerator
from autocommit_evaluation.cmg import evaluator
from autocommit_evaluation.core import (
    open_ai_few_shot_high_level_context_cmg_chain,
    open_ai_low_level_context_cmg_chain,
    open_ai_zero_shot_high_level_context_cmg_chain,
    open_ai_high_level_context_chain,
    deepseek_zero_shot_high_level_context_cmg_chain,
    deepseek_high_level_context_chain,
)
from autocommit.core.models import CommitDataModel
from autocommit_evaluation.datapreparation import context_generator, example_generator

KeyboardInterrupt: 

# Initialization

In [ ]:
COMMIT_DATA_JSON_FILE_PATH = os.path.join("autocommit_evaluation", "data", "cmg", "commits.json")
EXAMPLE_DATA_JSON_FILE_PATH = os.path.join("autocommit_evaluation", "data", "cmg", "commits.example.json")
CONTEXT_DATA_PATH = os.path.join("data", "context")

DEFAULT_CONTEXT_GENERATION_OUTPUT_PATH = os.path.join(
    "autocommit_evaluation", "data", "context"
)
DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.path.join(
    "out", "result", "highlevelcontext"
)
DEFAULT_CMG_OUTPUT_PATH = os.path.join("out", "result", "cmg")
DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH = os.path.join(
    "out", "result", "diffclassification"
)
DEFAULT_EXAMPLE_GENERATION_OUTPUT_PATH = os.path.join("out", "result", "example")

DIFF_CLASSIFIER_CHAINS = [
    open_ai_zero_shot_high_level_context_cmg_chain,
    open_ai_low_level_context_cmg_chain,
    deepseek_zero_shot_high_level_context_cmg_chain
]

HIGH_LEVEL_CONTEXT_CHAINS = [
    open_ai_high_level_context_chain,
    deepseek_high_level_context_chain,
]

GENERATORS = [
    CommitMessageGenerator(
        "Open AI Zero-Shot High-Level Context Generator", open_ai_zero_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator(
        "Open AI Few-Shot High-Level Context Generator", open_ai_few_shot_high_level_context_cmg_chain
    ),
    CommitMessageGenerator("Open AI Low-Level Context Generator", open_ai_low_level_context_cmg_chain),
    CommitMessageGenerator(
        "DeepSeek Zero-Shot High-Level Context Generator", deepseek_zero_shot_high_level_context_cmg_chain
    )
]

In [ ]:
CONTEXT_GENERATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CONTEXT_GENERATION_OUTPUT_PATH.value,
        DEFAULT_CONTEXT_GENERATION_OUTPUT_PATH,
    )

HIGH_LEVEL_CONTEXT_OUTPUT_PATH = os.getenv(
        EnvironmentKey.HIGH_LEVEL_CONTEXT_OUTPUT_PATH.value,
        DEFAULT_HIGH_LEVEL_CONTEXT_OUTPUT_PATH,
    )

CMG_OUTPUT_PATH = os.getenv(
        EnvironmentKey.CMG_OUTPUT_PATH.value, DEFAULT_CMG_OUTPUT_PATH
    )

DIFF_CLASSIFICATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.DIFF_CLASSIFICATION_OUTPUT_PATH.value,
        DEFAULT_DIFF_CLASSIFICATION_OUTPUT_PATH,
    )

EXAMPLE_GENERATION_OUTPUT_PATH = os.getenv(
        EnvironmentKey.EXAMPLE_GENERATION_OUTPUT_PATH.value,
        DEFAULT_EXAMPLE_GENERATION_OUTPUT_PATH,
    )

In [ ]:
def get_commits(path: str) -> list[CommitDataModel]:
        with open(path, "r", encoding="utf-8") as file:
            json_string = file.read()

        return CommitDataModel.from_json(json_string)

COMMITS = get_commits(COMMIT_DATA_JSON_FILE_PATH)
EXAMPLE_COMMITS = get_commits(EXAMPLE_DATA_JSON_FILE_PATH)

# Generate Context

In [ ]:
context_generator.generate_context(COMMITS + EXAMPLE_COMMITS, CONTEXT_GENERATION_OUTPUT_PATH)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 229
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/KAFKA-18401 HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 229
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/KAFKA-806 HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 229
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/KAFKA-17561 HTTP/11" 200 2534
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): is

# Generate Examples

In [ ]:
example_generator.generate_examples(EXAMPLE_COMMITS, EXAMPLE_GENERATION_OUTPUT_PATH)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 229
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/HADOOP-12657 HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 None
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/AMQ-6894 HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): issues.apache.org:443
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/serverInfo HTTP/11" 200 None
DEBUG:urllib3.connectionpool:https://issues.apache.org:443 "GET /jira/rest/api/2/issue/CASSANDRA-20188 HTTP/11" 200 None


# Generate Commit Message

In [ ]:
evaluator.evaluate(GENERATORS, COMMITS, CONTEXT_DATA_PATH, CMG_OUTPUT_PATH)